In [1]:
import pandas as pd
import random
from sklearn.metrics import accuracy_score

In [2]:
target = pd.read_csv('../data/target.csv')
predictors = pd.read_csv('../data/predictors.csv')

In [3]:
trucks = target[['EquipmentID']].drop_duplicates()['EquipmentID'].to_list()

random.shuffle(trucks)

In [4]:
trainTrucks = trucks[0:831]

testTrucks = trucks[831:]

In [5]:
xTrain = predictors.loc[predictors['EquipmentID'].isin(trainTrucks)].drop(columns = 'EquipmentID')
xTest = predictors.loc[predictors['EquipmentID'].isin(testTrucks)].drop(columns = 'EquipmentID')
yTrain = target.loc[target['EquipmentID'].isin(trainTrucks)].drop(columns = 'EquipmentID')
yTest = target.loc[target['EquipmentID'].isin(testTrucks)].drop(columns = 'EquipmentID')

In [6]:
from sklearn.neural_network import MLPClassifier

In [7]:
nn = MLPClassifier(activation = 'tanh', hidden_layer_sizes = (100, 100, 100), alpha = 1, max_iter = 10000)

nn.fit(xTrain, yTrain['target'] == 1)

MLPClassifier(activation='tanh', alpha=1, hidden_layer_sizes=(100, 100, 100),
              max_iter=10000)

In [22]:
yTest['prediction'] = nn.predict_proba(xTest)[:, 1]

In [23]:
yTest

,target,prediction
0,0.0,0.001436
1,0.0,0.001436
2,0.0,0.001436
3,0.0,0.001436
4,0.0,0.001436
...,...,...
546059,0.0,0.001436
546060,0.0,0.001436
546061,0.0,0.001436
546062,0.0,0.001436


In [17]:
yTrain.sum()

target    513.0
dtype: float64

In [18]:
yTest.sum()

target        140.000000
prediction    150.335438
dtype: float64

Look into over sampling